In [1]:
# general data analysis package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# file management package
import csv, sys
import os
import os.path
import copy
from tqdm.notebook import trange
import warnings
import pickle

# preprocessing onwards
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.feature_selection import f_classif

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, Normalizer, PowerTransformer, QuantileTransformer, RobustScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn import tree

#from sklearn.exceptions import ConvergenceWarning
from sklearn.feature_selection import SelectFromModel
from sklearn.inspection import permutation_importance

In [3]:
cd ../data

/Users/nadawahyu/Documents/Uni/Master/Forschungspraktikum_2/data


In [4]:
#data_pa = pd.read_csv('data_allo_pchem.csv')
#data_pb = pd.read_csv('data_binding_pchem.csv')
#data_sa = pd.read_csv('data_allo_substructure.csv')
#data_sb = pd.read_csv('data_binding_substructure.csv')

data_da = pd.read_csv('data_allo_desc.csv')
#data_db = pd.read_csv('data_binding_desc.csv')

#data_fa = pd.read_csv('data_allo_fp.csv')#.drop(['Unnamed: 0'], axis=1)
#data_fa = data_fa[data_fa['LigandType']!='none']
#data_fb = pd.read_csv('data_binding_fp.csv')#.drop(['Unnamed: 0'], axis=1)
#data_fb = data_fb[data_fb['LigandType']!='none']

In [5]:
# read pickle
#X_train_pa, X_val_pa, X_test_pa, y_train_pa, y_val_pa, y_test_pa, descnm_pa = pd.read_pickle('X_pa.pkl')
#X_train_pb, X_val_pb, X_test_pb, y_train_pb, y_val_pb, y_test_pb, descnm_pb = pd.read_pickle('X_pb.pkl')
#X_train_sa, X_val_sa, X_test_sa, y_train_sa, y_val_sa, y_test_sa, descnm_sa = pd.read_pickle('X_sa.pkl')
#X_train_sb, X_val_sb, X_test_sb, y_train_sb, y_val_sb, y_test_sb, descnm_sb = pd.read_pickle('X_sb.pkl')
X_train_da, X_val_da, X_test_da, y_train_da, y_val_da, y_test_da, descnm_da = pd.read_pickle('X_da.pkl')
#X_train_db, X_val_db, X_test_db, y_train_db, y_val_db, y_test_db, descnm_db = pd.read_pickle('X_db.pkl')
#X_train_fa, X_val_fa, X_test_fa, y_train_fa, y_val_fa, y_test_fa, descnm_fa = pd.read_pickle('X_fa.pkl')
#X_train_fb, X_val_fb, X_test_fb, y_train_fb, y_val_fb, y_test_fb, descnm_fb = pd.read_pickle('X_fb.pkl')

# da

## 6:2:2

In [6]:
target='LigandType'
col_to_drop=['lig_smiles', 'lig_at']
test_size=0.2
val_size = 0.2
random_state=0

data = data_da.copy()
data_ = data.copy()

# drop duplicate rows
data_ = data.drop_duplicates().dropna()

# drop cols with 'unnamed'
colidx_to_drop = [i for i in data.columns if 'Unnamed' in i]

#encode target
to_mask = {}

for i,j in enumerate(data_[target].unique()):
  to_mask.update({j:i})

data_ = data_.replace(to_mask)

# separate x and y
X = data_.loc[:, data_.columns.difference(col_to_drop+[target]+colidx_to_drop)]
y = data_[target]

# remove features that don't have variation
X_out = X.loc[:,X.apply(pd.Series.nunique) != 1]
mask = X_out.applymap(lambda x: isinstance(x, (int, float)))
X_out2 = X_out.where(mask)
X = X_out2.dropna(axis=1)

# anova feature selection
f_stat, p_val, = f_classif(X, y)

# get index which probability is lower than p = 0.05
idx = [i for i,j in enumerate(p_val) if j < 0.05]
# select column with respective index
X = X.iloc[:,idx]

# split train-validation and test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size = test_size, random_state=random_state, stratify=y)
# split train and validation
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size = val_size/(val_size+(1-val_size-test_size)), random_state=random_state, stratify=y_trainval)

descnm = np.array(X_train.columns)
#X_train = np.array(X_train)
#X_val = np.array(X_val)
#X_test = np.array(X_test)
#y_train = np.array(y_train)
#y_val = np.array(y_val)
#y_test = np.array(y_test)


In [7]:
print(len(X_train)/len(data_da))
print(len(X_val)/len(data_da))
print(len(X_test)/len(data_da))

0.5998962117280747
0.20005189413596264
0.20005189413596264


In [8]:
#X_train, y_train, X_val, y_val = X_train_da, y_train_da, X_val_da, y_val_da
random_state = 0

X_train, y_train  = SMOTE(random_state=random_state).fit_resample(X_train, y_train)

rfc = RandomForestClassifier(random_state=random_state)
fit = rfc.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 1.000
f1: 1.000
balanced accuracy: 1.000
matthews corrcoeff: 1.000

val scores 

accuracy: 0.904
f1: 0.948
balanced accuracy: 0.604
matthews corrcoeff: 0.288


In [9]:
print(classification_report(y_train, y_pred))
print(confusion_matrix(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2095
           1       1.00      1.00      1.00      2095

    accuracy                           1.00      4190
   macro avg       1.00      1.00      1.00      4190
weighted avg       1.00      1.00      1.00      4190

[[2095    0]
 [   0 2095]]


In [10]:
print(classification_report(y_val, y_val_pred))
print(confusion_matrix(y_val, y_val_pred))

              precision    recall  f1-score   support

           0       0.47      0.24      0.31        72
           1       0.93      0.97      0.95       699

    accuracy                           0.90       771
   macro avg       0.70      0.60      0.63       771
weighted avg       0.88      0.90      0.89       771

[[ 17  55]
 [ 19 680]]


In [11]:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4, 5, 6, 7, 8, 9, 10],
    'min_samples_leaf': [1, 5, 10],
    'criterion' :['gini', 'entropy'],
}

In [10]:
CV_rfc = GridSearchCV(estimator=rfc,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

Fitting 10 folds for each of 336 candidates, totalling 3360 fits


[CV 1/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.637 total time=   0.4s
[CV 2/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.828 total time=   0.5s
[CV 3/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.819 total time=   0.4s
[CV 4/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.814 total time=   0.4s
[CV 5/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.850 total time=   0.2s
[CV 6/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.854 total time=   0.4s
[CV 7/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.792 total time=   0.2s
[CV 8/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=0),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 5, 10],
                         'n_estimators': [10, 50, 100, 200]},
             verbose=3)

In [12]:
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

{'criterion': 'gini', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 1, 'n_estimators': 200}
RandomForestClassifier(max_depth=10, max_features='log2', n_estimators=200,
                       random_state=0)


In [49]:
rfc = RandomForestClassifier(n_estimators=200,
                             #min_samples_split=2,
                             min_samples_leaf=1,
                             max_features='log2',
                             max_depth=10,
                             criterion='gini',
                             )
fit = rfc.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.976
f1: 0.975
balanced accuracy: 0.976
matthews corrcoeff: 0.953

val scores 

accuracy: 0.850
f1: 0.916
balanced accuracy: 0.599
matthews corrcoeff: 0.184


### paperlike

In [12]:
param_grid = {
    #'n_estimators': [10, 50, 100, 200],
    #'max_features': ['sqrt', 'log2'],
    #'max_depth' : [4, 5, 6, 7, 8, 9, 10],
    'min_samples_leaf': [1, 5, 10],
    #'criterion' :['gini', 'entropy'],
}

rfc = RandomForestClassifier(bootstrap=True,
                             n_estimators=100,
                             max_features='sqrt')
CV_rfc = GridSearchCV(estimator=rfc,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

print('')
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

Fitting 10 folds for each of 3 candidates, totalling 30 fits
[CV 1/10] END ...............min_samples_leaf=1;, score=0.819 total time=   5.3s
[CV 2/10] END ...............min_samples_leaf=1;, score=0.983 total time=   4.0s
[CV 3/10] END ...............min_samples_leaf=1;, score=0.976 total time=   6.1s
[CV 4/10] END ...............min_samples_leaf=1;, score=0.993 total time=   6.6s
[CV 5/10] END ...............min_samples_leaf=1;, score=0.990 total time=   5.5s
[CV 6/10] END ...............min_samples_leaf=1;, score=0.988 total time=   4.3s
[CV 7/10] END ...............min_samples_leaf=1;, score=0.990 total time=   4.2s
[CV 8/10] END ...............min_samples_leaf=1;, score=0.990 total time=   3.7s
[CV 9/10] END ...............min_samples_leaf=1;, score=0.983 total time=   3.6s
[CV 10/10] END ..............min_samples_leaf=1;, score=0.974 total time=   6.4s
[CV 1/10] END ...............min_samples_leaf=5;, score=0.749 total time=   3.6s
[CV 2/10] END ...............min_samples_leaf=5;

In [14]:
rfc = RandomForestClassifier(n_estimators=100,
                             class_weight='balanced_subsample',
                             min_samples_leaf=1,
                             max_features='sqrt',
                             max_depth=10,
                             bootstrap=True
                             )
fit = rfc.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.977
f1: 0.976
balanced accuracy: 0.977
matthews corrcoeff: 0.955

val scores 

accuracy: 0.851
f1: 0.917
balanced accuracy: 0.613
matthews corrcoeff: 0.205


In [ ]:
train scores 

accuracy: 0.976
f1: 0.975
balanced accuracy: 0.976
matthews corrcoeff: 0.953

val scores 

accuracy: 0.850
f1: 0.916
balanced accuracy: 0.599
matthews corrcoeff: 0.184

## 4:2:4

In [51]:
target='LigandType'
col_to_drop=['lig_smiles', 'lig_at']
test_size=0.4
val_size = 0.2
random_state=0

data = data_da.copy()
data_ = data.copy()

# drop duplicate rows
data_ = data.drop_duplicates().dropna()

# drop cols with 'unnamed'
colidx_to_drop = [i for i in data.columns if 'Unnamed' in i]

#encode target
to_mask = {}

for i,j in enumerate(data_[target].unique()):
  to_mask.update({j:i})

data_ = data_.replace(to_mask)

# separate x and y
X = data_.loc[:, data_.columns.difference(col_to_drop+[target]+colidx_to_drop)]
y = data_[target]

# remove features that don't have variation
X_out = X.loc[:,X.apply(pd.Series.nunique) != 1]
mask = X_out.applymap(lambda x: isinstance(x, (int, float)))
X_out2 = X_out.where(mask)
X = X_out2.dropna(axis=1)

# anova feature selection
f_stat, p_val, = f_classif(X, y)

# get index which probability is lower than p = 0.05
idx = [i for i,j in enumerate(p_val) if j < 0.05]
# select column with respective index
X = X.iloc[:,idx]

# split train-validation and test
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size = test_size, random_state=random_state, stratify=y)
# split train and validation
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size = val_size/(val_size+(1-val_size-test_size)), random_state=random_state, stratify=y_trainval)

descnm = np.array(X_train.columns)
#X_train = np.array(X_train)
#X_val = np.array(X_val)
#X_test = np.array(X_test)
#y_train = np.array(y_train)
#y_val = np.array(y_val)
#y_test = np.array(y_test)


In [52]:
print(len(X_train)/len(data_da))
print(len(X_val)/len(data_da))
print(len(X_test)/len(data_da))

0.3998443175921121
0.20005189413596264
0.4001037882719253


In [53]:
rfc = RandomForestClassifier()
fit = rfc.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 1.000
f1: 1.000
balanced accuracy: 1.000
matthews corrcoeff: 1.000

val scores 

accuracy: 0.911
f1: 0.953
balanced accuracy: 0.540
matthews corrcoeff: 0.214


### gridsearch

In [62]:
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4, 5, 6, 7, 8, 9, 10],
    'min_samples_leaf': [1, 5, 10],
    'criterion' :['gini', 'entropy'],
}

In [63]:
rfc = RandomForestClassifier()
CV_rfc = GridSearchCV(estimator=rfc,
                      param_grid=param_grid,
                      cv= 10,
                      verbose=3)
CV_rfc.fit(X_train, y_train)

Fitting 10 folds for each of 336 candidates, totalling 3360 fits
[CV 1/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.903 total time=   0.2s


[CV 2/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.909 total time=   0.2s
[CV 3/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.909 total time=   0.1s
[CV 4/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.909 total time=   0.5s
[CV 5/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.909 total time=   0.2s
[CV 6/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.909 total time=   0.3s
[CV 7/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.903 total time=   0.4s
[CV 8/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n_estimators=10;, score=0.896 total time=   0.2s
[CV 9/10] END criterion=gini, max_depth=4, max_features=sqrt, min_samples_leaf=1, n

KeyboardInterrupt: 

In [ ]:
print(CV_rfc.best_params_)
print(CV_rfc.best_estimator_)

{'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 50}
RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=50)


In [59]:
rfc = RandomForestClassifier(n_estimators=200,
                             #min_samples_split=2,
                             min_samples_leaf=1,
                             max_features='log2',
                             max_depth=10,
                             criterion='gini',
                             )
fit = rfc.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.941
f1: 0.968
balanced accuracy: 0.686
matthews corrcoeff: 0.591

val scores 

accuracy: 0.905
f1: 0.950
balanced accuracy: 0.499
matthews corrcoeff: -0.012


In [57]:
rfc = RandomForestClassifier(n_estimators=50,
                             #min_samples_split=2,
                             min_samples_leaf=1,
                             max_features='sqrt',
                             max_depth=10,
                             criterion='entropy',
                             bootstrap=True
                             )
fit = rfc.fit(X_train, y_train)

y_pred = fit.predict(X_train)
y_val_pred = fit.predict(X_val)

# calculate statistical metrics accuracy for training set
train_accuracy = metrics.accuracy_score(y_train, y_pred)
val_accuracy = metrics.accuracy_score(y_val, y_val_pred)

# calculate statistical metrics for training set
train_f1 = metrics.f1_score(y_train, y_pred, average = 'binary')
train_ba = metrics.balanced_accuracy_score(y_train, y_pred)
train_mcc = metrics.matthews_corrcoef(y_train, y_pred)

# calculate statistical metrics for val set
val_f1 = metrics.f1_score(y_val, y_val_pred, average = 'binary')
val_ba = metrics.balanced_accuracy_score(y_val, y_val_pred)
val_mcc = metrics.matthews_corrcoef(y_val, y_val_pred)

print(f'train scores \n')
print(f'accuracy: {train_accuracy:.3f}')
print(f'f1: {train_f1:.3f}')
print(f'balanced accuracy: {train_ba:.3f}')
print(f'matthews corrcoeff: {train_mcc:.3f}')
print('')
print(f'val scores \n')
print(f'accuracy: {val_accuracy:.3f}')
print(f'f1: {val_f1:.3f}')
print(f'balanced accuracy: {val_ba:.3f}')
print(f'matthews corrcoeff: {val_mcc:.3f}')

train scores 

accuracy: 0.942
f1: 0.969
balanced accuracy: 0.693
matthews corrcoeff: 0.603

val scores 

accuracy: 0.904
f1: 0.950
balanced accuracy: 0.499
matthews corrcoeff: -0.016
